# 執行腳本

In [1]:
%load_ext autoreload
%autoreload 2

import data_loader
import data_preprocessing
import feature_eng
import model_train
# import experiment_logger

# ------------------------------------------------
# Step 1: 載入 (Load) - 只管讀進來
# ------------------------------------------------
df_raw = data_loader.load_and_merge_data()

# ------------------------------------------------
# Step 2: 前處理 (Preprocess) - 只管洗乾淨
# ------------------------------------------------
df_clean = data_preprocessing.clean_data(df_raw)

# ------------------------------------------------
# Step 3: 特徵工程 (Feature) - 只管算指標
# ------------------------------------------------
# 注意：用洗乾淨的資料來算指標，才不會報錯
df_features = feature_eng.add_technical_indicators(df_clean)

# ------------------------------------------------
# Step 4: 訓練並產生交卷檔案
# ------------------------------------------------
model = model_train.train_and_predict(df_features, submission_file='data/sample_submission.csv')
print("🎉 整個流程執行完畢！")
print("\n✅ 全流程結束！請去資料夾找 submission.csv 上傳！")


🚀 [Loader] 開始讀取資料...
✅ 資料載入完成 (尚未清洗)！大小：(852, 12)
🧹 [Preprocessing] 開始清洗資料...
✅ 資料清洗完成！沒有 NaN 了。
📊 [Feature Engineering] 偵測到 11 支股票，開始計算全套技術指標...
✅ 特徵工程完成！目前的欄位數: 122
🚀 [Training] 啟動模型訓練生產線...
📚 訓練資料集: (695, 120)
📝 預測資料集: (157, 120) (應與 sample_submission 列數相同)
🤝 正在組建集成模型 (XGBoost + Random Forest + LightGBM)...
🏋️ 開始訓練模型 (這可能需要幾秒鐘)...


/Users/alice/Downloads/人工智慧/stock_project/feature_eng.py:101: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{prefix}_Vol_20'] = df[col].pct_change().rolling(window=20).std()
/Users/alice/Downloads/人工智慧/stock_project/feature_eng.py:110: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{prefix}_BBP'] = (df[col] - lower_band) / (upper_band - lower_band + 1e-9)
/Users/alice/Downloads/人工智慧/stock_project/feature_eng.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` 

✅ 模型訓練完成！
🔮 正在進行最終預測...
🎉 恭喜！考卷已填寫完成，檔案位於: submission.csv
前 5 筆預測結果預覽：
   date  0056_close_y
0  8086    374.856899
1  8085    374.933320
2  8084    374.937935
3  8083    374.933276
4  8082    374.388610
🎉 整個流程執行完畢！

✅ 全流程結束！請去資料夾找 submission.csv 上傳！


## 3. 資料品質驗收 (Data Quality Check)

在進入模型訓練前，我們對特徵工程後的資料進行最終檢查，確保：
1. 資料維度正確。
2. 無任何遺漏值 (NaN)。
3. 特徵欄位已正確生成。

In [2]:
# --- 資料健康檢查 ---
print("🧐 [Data Check] 正在檢查 df_features 資料品質...")

# 1. 檢查是否有空值 (Critical Check)
nan_count = df_features.isnull().sum().sum()
if nan_count == 0:
    print("✅ 通過檢查：資料集中已無任何空值 (NaN count = 0)。")
else:
    print(f"⚠️ 警告：偵測到 {nan_count} 個空值，請回頭檢查 data_preprocessing 或 feature_eng！")

# 2. 檢查資料規模
print(f"📊 資料規模: {df_features.shape[0]} 列 x {df_features.shape[1]} 欄")

# 3. 預覽數據 (隨機抽樣 3 筆)
print("\n👀 資料預覽 (Sample 3 rows):")
# 只顯示前 10 個欄位以免版面太亂，中間用 ... 帶過
display_cols = df_features.columns[:5].tolist() + ['...'] + df_features.columns[-5:].tolist()
print(df_features.sample(3).to_string()) # to_string() 可以避免 jupyter 顯示跑版

🧐 [Data Check] 正在檢查 df_features 資料品質...
✅ 通過檢查：資料集中已無任何空值 (NaN count = 0)。
📊 資料規模: 852 列 x 122 欄

👀 資料預覽 (Sample 3 rows):
     date  0056_close_y  1101_close  2327_close  2352_close  2385_close  2449_close  2915_close  3005_close  3532_close  6176_close  9945_close   0056_MA_5   0056_RSI  0056_MACD_DIF  0056_MACD_DEM  0056_MACD_OSC  0056_Bias_5  0056_Bias_20  0056_Return_1  0056_Vol_20  0056_BBP  1101_MA_5   1101_RSI  1101_MACD_DIF  1101_MACD_DEM  1101_MACD_OSC  1101_Bias_5  1101_Bias_20  1101_Return_1  1101_Vol_20  1101_BBP  2327_MA_5   2327_RSI  2327_MACD_DIF  2327_MACD_DEM  2327_MACD_OSC  2327_Bias_5  2327_Bias_20  2327_Return_1  2327_Vol_20  2327_BBP  2352_MA_5   2352_RSI  2352_MACD_DIF  2352_MACD_DEM  2352_MACD_OSC  2352_Bias_5  2352_Bias_20  2352_Return_1  2352_Vol_20  2352_BBP  2385_MA_5   2385_RSI  2385_MACD_DIF  2385_MACD_DEM  2385_MACD_OSC  2385_Bias_5  2385_Bias_20  2385_Return_1  2385_Vol_20  2385_BBP  2449_MA_5   2449_RSI  2449_MACD_DIF  2449_MACD_DEM  2449_MACD_OSC  2449_